In [1]:
import cv2
import numpy as np
import glob as gb
import pandas as pd

ModuleNotFoundError: No module named 'cv2'

In [ ]:
image_path = gb.glob('Hoda 0-9/'+"*.bmp")
image_path

['Hoda 0-9\\0_1007.bmp',
 'Hoda 0-9\\0_1016.bmp',
 'Hoda 0-9\\0_1033.bmp',
 'Hoda 0-9\\0_1037.bmp',
 'Hoda 0-9\\0_1040.bmp',
 'Hoda 0-9\\0_1042.bmp',
 'Hoda 0-9\\0_1043.bmp',
 'Hoda 0-9\\0_1048.bmp',
 'Hoda 0-9\\0_1051.bmp',
 'Hoda 0-9\\0_1054.bmp',
 'Hoda 0-9\\0_1057.bmp',
 'Hoda 0-9\\0_106.bmp',
 'Hoda 0-9\\0_107.bmp',
 'Hoda 0-9\\0_1072.bmp',
 'Hoda 0-9\\0_1080.bmp',
 'Hoda 0-9\\0_1082.bmp',
 'Hoda 0-9\\0_1089.bmp',
 'Hoda 0-9\\0_1094.bmp',
 'Hoda 0-9\\0_1097.bmp',
 'Hoda 0-9\\0_1105.bmp',
 'Hoda 0-9\\0_1113.bmp',
 'Hoda 0-9\\0_1116.bmp',
 'Hoda 0-9\\0_1120.bmp',
 'Hoda 0-9\\0_1135.bmp',
 'Hoda 0-9\\0_114.bmp',
 'Hoda 0-9\\0_1141.bmp',
 'Hoda 0-9\\0_1150.bmp',
 'Hoda 0-9\\0_1151.bmp',
 'Hoda 0-9\\0_1165.bmp',
 'Hoda 0-9\\0_1167.bmp',
 'Hoda 0-9\\0_1177.bmp',
 'Hoda 0-9\\0_1179.bmp',
 'Hoda 0-9\\0_1184.bmp',
 'Hoda 0-9\\0_1189.bmp',
 'Hoda 0-9\\0_1200.bmp',
 'Hoda 0-9\\0_1208.bmp',
 'Hoda 0-9\\0_1219.bmp',
 'Hoda 0-9\\0_1235.bmp',
 'Hoda 0-9\\0_1242.bmp',
 'Hoda 0-9\\0_1243.bmp',
 'H

In [ ]:
x = []
y = []

# Feature and Label Extraction

In [ ]:
for i in range(len(image_path)):
    path = image_path[i]
    idx = path.rfind('_') - 1
    y.append(path[idx])
    img = cv2.imread(path, 0)
    img = cv2.resize(img, dsize=(10, 10))
    #cv2.imshow('img', img)
    #ch = cv2.waitKey(0)
    #if ch == ord('q'):
        #break
    data = np.reshape(img, newshape=(100,))
    x.append(data)

# Shuffle and Split

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
df = pd.DataFrame(x)
df['label'] = y

In [ ]:
def shuffle_and_extract(df, label_index=-1, seed=42):
    np.random.seed(seed)
    df = df.sample(frac=1).reset_index(drop=True)
    x = df.iloc[:, :label_index]
    y = df.iloc[:, label_index]
    return x, y

In [ ]:
x, y = shuffle_and_extract(df)
x = np.array(x)
y = np.array(y)

In [ ]:
train_test_index = int(0.7* len(x))
x_train = x[0:train_test_index, :]
y_train = y[0:train_test_index]

x_test = x[train_test_index:, :]
y_test = y[train_test_index:]

# Normalization

In [ ]:
x_train = x_train/255
x_test = x_test/255

# MLP classifier
using sklearn models

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(50, 20), (100, 50), (150, 100)],
    'activation': ['logistic', 'tanh', 'relu'],
    'tol': [0.000001, 0.0001, 0.001],
    'momentum': [0.9, 0.95, 0.99],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01]
}

In [ ]:
mlp = MLPClassifier(verbose=False)
grid_search = GridSearchCV(mlp, param_grid, scoring='accuracy', n_jobs=-1, cv=5)

Fitting The model

In [ ]:
grid_search.fit(x_train, y_train)

KeyboardInterrupt: 

In [ ]:
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

In [ ]:
mlp = MLPClassifier(verbose=True, activation='tanh', alpha=0.01, hidden_layer_sizes=(100, 50), momentum=0.95, solver='lbfgs', tol=1e-06, max_iter=100000000)

In [ ]:
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.91        47
           1       0.87      0.96      0.91        47
           2       0.88      0.88      0.88        42
           3       0.92      0.94      0.93        49
           4       0.93      0.77      0.84        56
           5       0.94      0.87      0.90        54
           6       0.85      0.88      0.86        57
           7       0.84      0.91      0.87        45
           8       0.94      0.95      0.95        65
           9       0.92      0.92      0.92        48

    accuracy                           0.90       510
   macro avg       0.90      0.90      0.90       510
weighted avg       0.90      0.90      0.90       510


In [ ]:
import joblib

In [ ]:
joblib.dump(mlp, 'mlp_network.joblib')

['mlp_network.joblib']

In [ ]:
new_mlp = joblib.load('mlp_network.joblib')

In [ ]:
print(new_mlp.score(x_train, y_train))
print(new_mlp.score(x_test, y_test))

1.0
0.8980392156862745
